<a href="https://colab.research.google.com/github/BiancadeFrancisco/DecisionTreeClassifier_Tunned_Churn/blob/main/Tunning_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel("/content/churn_data.xlsx")

df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer,42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
# COLUNA TOTALCHARGES ESTÁ COMO OBJECT, TRANSFORMAR EM NUMERICA:

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

In [ ]:
df = df.dropna()

In [ ]:
# SEPARAR VARIÁVEIS:

cat_vars = ["PhoneService", "Contract", "PaperlessBilling", "PaymentMethod"] #VARIÁVEL CATEGÓRICA

num_vars = ["tenure", "MonthlyCharges", "TotalCharges"] #VARIÁVEL NUMÉRICA

modeling_vars = cat_vars + num_vars

In [ ]:
# DEFINIR X E Y:

X = df.filter(modeling_vars).copy()

y = df["Churn"].copy()

y = pd.DataFrame(y)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(y.Churn)

y.Churn = le.transform(y.Churn)

In [ ]:
X =   pd.get_dummies(X)
X.head()

,tenure,MonthlyCharges,TotalCharges,PhoneService_No,PhoneService_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer,PaymentMethod_Credit card,PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,29.85,29.85,1,0,1,0,0,0,1,0,0,1,0
1,34,56.95,1889.50,0,1,0,1,0,1,0,0,0,0,1
2,2,53.85,108.15,0,1,1,0,0,0,1,0,0,0,1
3,45,42.30,1840.75,1,0,0,1,0,1,0,1,0,0,0
4,2,70.70,151.65,0,1,1,0,0,0,1,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [ ]:
# FAZER TUNNING DO MODELO:



# 1º: GridSerch

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier()

parameters = {
    'max_depth': [1,2,3,4,5,6,7,8,9,10]
}

grid_search = GridSearchCV(arvore, parameters, scoring="roc_auc", cv=5, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='roc_auc')

In [ ]:
GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
                param_grid = {'max_depth': [1,2,3,4,5,6,7,8,9,10]},
                scoring = "roc_auc")

results = pd.DataFrame(grid_search.cv_results_)

In [ ]:
results.sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,0.019821,0.001528,0.013474,0.000680,5,{'max_depth': 5},0.826368,0.805496,0.832924,0.823451,0.839897,0.825627,0.011554,1
5,0.021294,0.000610,0.015446,0.003914,6,{'max_depth': 6},0.822979,0.810468,0.829243,0.824603,0.834120,0.824283,0.007925,2
3,0.017598,0.001098,0.013560,0.000748,4,{'max_depth': 4},0.830318,0.802204,0.828877,0.812071,0.832328,0.821160,0.011917,3
6,0.022162,0.000322,0.012488,0.000512,7,{'max_depth': 7},0.817369,0.803206,0.807241,0.816080,0.823886,0.813556,0.007411,4
2,0.015739,0.001252,0.016608,0.006196,3,{'max_depth': 3},0.822973,0.786206,0.824998,0.782489,0.822355,0.807804,0.019208,5
7,0.023349,0.000285,0.014132,0.001853,8,{'max_depth': 8},0.796794,0.793470,0.779068,0.789508,0.822404,0.796249,0.014372,6
8,0.026997,0.003553,0.015944,0.006628,9,{'max_depth': 9},0.771186,0.780408,0.765575,0.765089,0.808450,0.778142,0.016127,7
1,0.014882,0.003895,0.013264,0.001398,2,{'max_depth': 2},0.779175,0.753401,0.779989,0.766237,0.804794,0.776719,0.017079,8
9,0.027135,0.001005,0.012657,0.000470,10,{'max_depth': 10},0.756938,0.750956,0.749743,0.749808,0.788152,0.759119,0.014758,9
0,0.011714,0.001281,0.014843,0.002952,1,{'max_depth': 1},0.730212,0.713838,0.726354,0.716691,0.746780,0.726775,0.011673,10


In [ ]:
# TESTAR COM RANDOM FOREST:

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

In [ ]:
parameters = {"max_depth": [1,2,3,4,5,6,7,8,9,10],
              "n_estimators": [100, 300, 500]}

grid_search = GridSearchCV(rf, parameters, scoring="roc_auc", cv=5, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 300, 500]},
             scoring='roc_auc')

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by="rank_test_score").head()


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
23,3.497940,0.594001,0.152180,0.004036,8,500,"{'max_depth': 8, 'n_estimators': 500}",0.842125,0.822129,0.854244,0.834128,0.858914,0.842308,0.013359,1
20,3.900604,1.065233,0.145005,0.007094,7,500,"{'max_depth': 7, 'n_estimators': 500}",0.842927,0.819218,0.856697,0.831211,0.860170,0.842045,0.015378,2
19,1.804602,0.024227,0.088432,0.001398,7,300,"{'max_depth': 7, 'n_estimators': 300}",0.842587,0.819021,0.855503,0.830800,0.859919,0.841566,0.015207,3
22,2.196837,0.592133,0.112229,0.035859,8,300,"{'max_depth': 8, 'n_estimators': 300}",0.841558,0.821313,0.854442,0.832424,0.857422,0.841432,0.013497,4
17,3.400570,0.660613,0.162918,0.050127,6,500,"{'max_depth': 6, 'n_estimators': 500}",0.843145,0.817473,0.855039,0.829972,0.860174,0.841161,0.015776,5


In [ ]:
grid_search.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# TESTAR COM LIGHT GBM:

from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {"learning_rate": [0.001, 0.01],
              "num_leaves": [2, 128],
              "min_child_samples": [1,100],
              "subsample": [0.05, 1],
              "colsample_bytree": [0.1, 1.0]}

random_search = RandomizedSearchCV(lgbm, parameters, scoring="roc_auc", cv=5, n_iter=5, n_jobs=-1)

random_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 1.0],
                                        'learning_rate': [0.001, 0.01],
                                        'min_child_samples': [1, 100],
                                        'num_leaves': [2, 128],
                                        'subsample': [0.05, 1]},
                   scoring='roc_auc')

In [ ]:
results = pd.DataFrame(random_search.cv_results_)
results.sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_num_leaves,param_min_child_samples,param_learning_rate,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,0.182256,0.009115,0.018142,0.001548,1,128,100,0.001,1.0,"{'subsample': 1, 'num_leaves': 128, 'min_child...",0.829539,0.808731,0.837785,0.825271,0.848329,0.829931,0.013202,1
3,0.178774,0.004459,0.017749,0.000472,0.05,128,100,0.001,1.0,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.829539,0.808731,0.837785,0.825271,0.848329,0.829931,0.013202,1
4,0.057755,0.001547,0.016068,0.000454,0.05,2,100,0.001,0.1,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",0.821119,0.794288,0.835867,0.800117,0.842791,0.818836,0.019090,3
1,0.112065,0.007670,0.019277,0.001489,0.05,128,1,0.001,0.1,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.810887,0.801572,0.832733,0.793701,0.849204,0.817620,0.020499,4
0,0.538904,0.024270,0.020398,0.001426,1,128,1,0.001,1.0,"{'subsample': 1, 'num_leaves': 128, 'min_child...",0.809160,0.803044,0.801665,0.808919,0.824824,0.809523,0.008226,5
